In [107]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 

In [108]:
# loading the data set
dataset = load_breast_cancer(as_frame=True)
df= pd.DataFrame(data= dataset.data)
df["target"] = dataset.target

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [109]:
# Seperating to X and Y 
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# splitting training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=24) 

In [110]:
def param_init(X): 
    """
    Initialize parameters
    __________________ 
    Input(s)
    X: Training data
    __________________
    Output(s)
    params: Dictionary containing coefficients
    """
    params = {} # initialize dictionary 
    _, n_features = X.shape # shape of training data

    # initializing coefficents to 0 
    params["W"] = np.zeros(n_features)
    params["b"] = 0
    return params 

In [111]:
def get_z(X, W, b): 
    """
    Calculates Linear Function
    __________________
    Input(s)
    X: Training data
    W: Weight coefficients
    b: bias coefficients
    __________________
    Output(s)
    z: a Linear function
    """
    z = np.dot(X, W) + b
    return z

In [112]:
def sigmoid(z):
    """
    Logit model
    _________________
    Input(s)
    z: Linear model 
    _________________
    Output(s)
    g: Logit function applied to linear model
    """
    g = 1 / (1 + np.exp(-z))
    return g 

In [113]:
def gradient_descent(X, y, params, lr, n_iter): 
    """
    Gradient descent to minimize cost function
    __________________ 
    Input(s)
    X: Training data
    y: Labels
    params: Dictionary contatining coefficients
    lr: learning rate
    __________________
    Output(s)
    params: Dictionary containing optimized coefficients
    """
    W = params["W"] 
    b = params["b"]
    m = X.shape[0] # number of training instances 

    for _ in range(n_iter): 
        # prediction with random weights
        g = sigmoid(get_z(X, W, b))
        # calculate the loss
        loss = -1/m * np.sum(y * np.log(g)) + (1 - y) * np.log(1 - g)
        # partial derivative of weights 
        dW = 1/m * np.dot(X.T, (g - y))
        db = 1/m * np.sum(g - y)
        # updates to coefficients
        W -= lr * dW
        b -= lr * db 
    
    params["W"] = W
    params["b"] = b
    return params 

In [114]:
def train(X, y, lr=0.01, n_iter=1000):
    """
    Train Linear Regression model with Gradient decent
    __________________ 
    Input(s)
    X: Training data
    y: Labels
    lr: learning rate
    n_iter: Number of iterations 
    __________________
    Output(s)
    params: Dictionary containing optimized coefficients
    """ 
    init_params = param_init(X)
    params = gradient_descent(X, y, init_params, lr, n_iter)
    return params

In [134]:
def predict(X_test, params):
    """
    Train Linear Regression model with Gradient decent
    __________________ 
    Input(s)
    X: Unseen data
    params: Dictionary contianing optimized weights from training
    __________________
    Output(s)
    prediction of model
    """  
    z = np.dot(X_test, params["W"]) + params["b"]
    y_pred = sigmoid(z) >= 0.5
    return y_pred.astype("int")

In [135]:
params = train(X_train, y_train) # train model
y_pred = predict(X_test, params) # inference

In [136]:
lr = LogisticRegression(C=0.01)
lr.fit(X_train, y_train)
sklearn_y_pred = lr.predict(X_test)

In [137]:
print(f"My Implementation: {accuracy_score(y_test, y_pred)}\nSklearn Implementation: {accuracy_score(y_test, sklearn_y_pred)}")

My Implementation: 0.9300699300699301
Sklearn Implementation: 0.9300699300699301


## Object Oriented Programming (OOB)

In [96]:
class LogReg(): 
    """
    Custom made Logistic Regression class
    """
    def __init__(self, lr=0.01, n_iter= 1000): 
        self.lr = lr
        self.n_iter = n_iter 
        self.params = {}
    
    def param_init(self, X_train): 
        """
        Initialize parameters for linear regression model
        __________________ 
        Input(s)
        X: Training data
        """
        _, n_features = self.X.shape # shape of training data

        # initializing coefficents to 0 
        self.params["W"] = np.zeros(n_features)
        self.params["b"] = 0
        return self

    def get_z(X, W, b): 
        """
        Calculates Linear Function
        __________________
        Input(s)
        X: Training data
        W: Weight coefficients
        b: bias coefficients
        __________________
        Output(s)
        z: a Linear function
        """
        z = np.dot(X, W) + b
        return z
        
    def sigmoid(z):
        """
        Logit model
        _________________
        Input(s)
        z: Linear model 
        _________________
        Output(s)
        g: Logit function applied to linear model
        """
        g = 1 / (1 + np.exp(-z))
        return g 
        

    def gradient_descent(self, X_train, y_train): 
        """
        Gradient descent to minimize cost function
        __________________ 
        Input(s)
        X: Training data
        y: Labels
        params: Dictionary contatining random coefficients
        alpha: Model learning rate
        __________________
        Output(s)
        params: Dictionary containing optimized coefficients
        """
        W = self.params["W"] 
        b = self.params["b"] 
        m = X_train.shape[0]

        for _ in range(self.n_iter): 
            # prediction with random weights
            g = sigmoid(get_z(X, W, b))
            # calculate the loss
            loss = -1/m * np.sum(y * np.log(g)) + (1 - y) * np.log(1 - g)
            # partial derivative of weights 
            dW = 1/m * np.dot(X.T, (g - y))
            db = 1/m * np.sum(g - y)
            # updates to coefficients
            W -= self.lr * dW
            b -= self.lr * db 
        
        self.params["W"] = W
        self.params["b"] = b
        return self

    def train(self, X_train, y_train):
        """
        Train Linear Regression model with Gradient decent
        __________________ 
        Input(s)
        X: Training data
        y: Labels
        alpha: Model learning rate
        n_iter: Number of iterations 
        __________________
        Output(s)
        params: Dictionary containing optimized coefficients
        """ 
        self.params = param_init(X_train)
        gradient_descent(X_train, y_train, self.params , self.lr, self.n_iter)
        return self 

    def predict(self, X_test):
        """
        Train Linear Regression model with Gradient decent
        __________________ 
        Input(s)
        X: Unseen data
        params: Dictionary contianing optimized weights from training
        __________________
        Output(s)
        y_preds: Predictions of model
        """  
        g = sigmoid(np.dot(X_test, self.params["W"]) + self.params["b"])
        return g 

In [97]:
logreg = LogReg()
logreg.train(X_train, y_train)
oop_y_pred = logreg.predict(X_test)

In [98]:
oop_y_pred == y_preds 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,